In [40]:
# import all nessecary libraries
import sklearn
import numpy as np
import pandas as pd
import xgboost as xgb
from plotly import express as px
from plotly import graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

print("Succesfully imported all libraries")

Succesfully imported all libraries


In [32]:
df = pd.read_csv("../../data/pecan/sim_score_and_yield_05_23_24.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df.drop(columns=["Unnamed: 0"], inplace=True)
df.to_csv("../../data/pecan/sim_score_and_yield_v2_05_23_24.csv", index=False)
df.shape

(198, 15)

In [33]:
df.head()

,city_names,yield,lat_sim,lon_sim,dew_point_sim,feels_like_sim,temp_min_sim,temp_max_sim,pressure_sim,humidity_sim,wind_speed_sim,wind_deg_sim,clouds_all_sim,weather_id_sim,Similarity Type
0,Doña Ana-Chaves County,1500.0,681.651657,1475.458784,3.106781e+03,2.171000e+03,1.916711e+03,2.039767e+03,1.921502e+03,1.087209e+04,1592.502058,8.106963e+04,2.390010e+04,5.529241e+04,Euclidean
1,Chaves County-Otero County,1500.0,529.858496,706.427416,3.137783e+03,4.317280e+03,4.291811e+03,3.715638e+03,2.202131e+03,1.087521e+04,1447.905650,7.552922e+04,2.282695e+04,5.997001e+04,Euclidean
2,Chaves County-Sierra County,1500.0,214.786840,1655.156863,3.763669e+03,2.065357e+03,1.873333e+03,1.880476e+03,1.810586e+03,1.163909e+04,1542.782991,8.392373e+04,2.380490e+04,5.433247e+04,Euclidean
3,Doña Ana-Chaves County,1500.0,429174.292464,928962.724008,1.385963e+06,9.923937e+05,9.067831e+05,9.791663e+05,8.977060e+05,5.034805e+06,771617.400000,3.668089e+07,9.198797e+06,1.035451e+07,Manhattan
4,Chaves County-Otero County,1500.0,333603.891336,444773.343672,1.568665e+06,2.480981e+06,2.477631e+06,2.113210e+06,1.173129e+06,5.447879e+06,715409.720000,3.334995e+07,8.923066e+06,1.272044e+07,Manhattan


In [43]:

x = df.drop(columns=["yield", 'city_names', 'Similarity Type'])
y = df["yield"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

param_grid = {
    'max_depth': [3, 4, 6, 9, 12],
    'eta': [0.01, 0.05, 0.1, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'n_estimators': [100, 150, 200, 300, 500],
    'alpha': [0, 0.1, 0.5, 1, 1.5, 2],
    'lambda': [0, 0.1, 0.5, 1, 1.5, 2]
}

xgb_reg = xgb.XGBRegressor(objective="reg:squarederror")

grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

xg_reg_best = xgb.XGBRegressor(**best_params)
xg_reg_best.fit(x_train, y_train)

preds_best = xg_reg_best.predict(x_test)
rmse_best = np.sqrt(mean_squared_error(y_test, preds_best))
print(f"RMSE: {rmse_best}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=preds_best, mode="markers", name="Predicted vs Actual"))
fig.update_layout(title="Predicted vs Actual Yield", xaxis_title="Actual Yield", yaxis_title="Predicted Yield")
fig.show()

Fitting 5 folds for each of 32400 candidates, totalling 162000 fits
Best parameters: {'alpha': 2, 'colsample_bytree': 1.0, 'eta': 0.01, 'lambda': 2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
RMSE: 477.9034396030153
